# SDNet



Conversational Question Answering is an exciting task that requires the model to read a passage and answers questions in dialogue. It is different from Machine Reading Comprehension, where the model reads a passage and answers questions in a stateless manner, i.e. it doesn’t use information from previous questions and answers. This new task expects the model to comprehend the passage, understand the context, and do coreference resolution.

SDNet is a contextualized attention based deep neural network that achieved State of the Art results in the challenging task of Conversational Question Answering. It makes use of inter attention and self-attention along with Recurrent BIdirectional LSTM layers.

To read about it more, please refer [this](https://analyticsindiamag.com/complete-guide-to-sdnet-contextualized-attention-based-deep-network-for-conversational-question-answering/) article.

# Code Implementation

Microsoft has made the code for SDNet model opensource.
We can clone this repository with

In [1]:
!git clone https://github.com/microsoft/SDNet.git

Cloning into 'SDNet'...
remote: Enumerating objects: 43, done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (43/43), done.


Let’s use data from https://stanfordnlp.github.io/coqa/ to train and test the model.

We need to download data,bert model and glove embeddings to train this model. Following are the commands to get these files.

In [2]:
!wget https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json

--2021-06-18 08:34:53--  https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json [following]
--2021-06-18 08:34:53--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49001836 (47M) [application/json]
Saving to: ‘coqa-train-v1.0.json’

coqa-train-v1.0.jso 100%[===================>]  46.73M  11.8MB/s    in 4.0s    

2021-06-18 08:34:57 (11.8 MB/s) - ‘coqa-train-v1.0.json’ saved [49001836/49001836]



In [3]:
!wget https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json

--2021-06-18 08:35:04--  https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json [following]
--2021-06-18 08:35:05--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9090845 (8.7M) [application/json]
Saving to: ‘coqa-dev-v1.0.json’

coqa-dev-v1.0.json  100%[===================>]   8.67M  37.5MB/s    in 0.2s    

2021-06-18 08:35:05 (37.5 MB/s) - ‘coqa-dev-v1.0.json’ saved [9090845/9090845]



In [4]:
%cp coqa-dev-v1.0.json /content/coqa/data/

cp: cannot create regular file '/content/coqa/data/': No such file or directory


In [5]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz

--2021-06-18 08:35:07--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.26.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.26.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407873900 (389M) [application/x-tar]
Saving to: ‘bert-base-uncased.tar.gz’

bert-base-uncased.t 100%[===================>] 388.98M  47.5MB/s    in 9.1s    

2021-06-18 08:35:16 (42.6 MB/s) - ‘bert-base-uncased.tar.gz’ saved [407873900/407873900]



In [6]:
!tar -xf bert-base-uncased.tar.gz

In [7]:
!mkdir bert-base-uncased

In [8]:
!cp /content/SDNet/conf /content/coqa/

cp: cannot create regular file '/content/coqa/': Not a directory


In [9]:
!mv  bert-base-uncased /content/coqa/

In [10]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2021-06-18 08:35:22--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-06-18 08:35:22--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-06-18 08:35:22--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [11]:
!unzip glove.840B.300d.zip

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


We need to arrange these files into the  directory structure.

In [ ]:
!python SDNet/main.py train coqa/conf

Using Cuda

SDNet Model Trainer
CoQA Preprocessing
Select command: train
Saving logs, model and evaluation in coqa/conf~/run_2
Load train_meta.msgpack...
-----------------------------------------------
Initializing model...
SDNet model

Using BERT
Loading BERT model...
Using BERT Large model
Loading BERT model from coqa/bert-base-uncased/
03/16/2021 03:35:58 - INFO - Models.Bert.modeling -   loading archive file coqa/bert-base-uncased/
03/16/2021 03:35:58 - INFO - Models.Bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Finished loading
Lock BERT's weights
BERT_LARGE
BERT dim: 1024 BERT_LAYERS: 24
BERT_LINEAR_COMBINE
/content/SDNet/Models/SDNet.py:82: UserWarning: nn.init.constant is now deprecat

This command will train using train data and predict results for dev data. But let’s see how to only predict without training the model. To do this, we need a pretrained model, test data and a config file. We have saved all of them from the previous step.

In [12]:
from SDNet.Models.SDNetTrainer import SDNetTrainer
from SDNet.Utils.Arguments import Arguments
conf_args = Arguments(conf_file)
opt = conf_args.readArguments()
opt['cuda'] = torch.cuda.is_available()
opt['confFile'] = conf_file
opt['datadir'] = os.path.dirname(conf_file)  # conf_file specifies where the data folder is

for key,val in cmdline_args.__dict__.items():
    if val is not None and key not in ['command', 'conf_file']:
        opt[key] = val
print(opt)
model = SDNetTrainer(opt)
predictions,confidence,pred_json = model.official(model_path,test_data)

ModuleNotFoundError: ignored

In [ ]:
model.official()

Following are the validation F1 scores obtained by SDNet model using various settings on CoQA dataset.